## An example implementation of tag search

This shows how to achieve a UI for tag search.

Its not ideal in these ways:
- we're missing an intersection op.
- we'll need to be sure the pattern vectorizes
- list of tag strings may not be the best representation for performance at scale
- we don't have a nice way to display list of strings as tags
- no centralized tag color control
- you should be able to click on a tag and filter the table down to that tag
- varbar construction of controls is still pretty odd

But it works!


In [ ]:
import weave
from weave.legacy.weave import weave_internal

In [ ]:
data = weave.save([
    {'x': 15, 'tags': ['a', 'b']},
    {'x': 23.9, 'tags': ['b', 'c']},
    {'x': -8, 'tags': ['a']}
], 'taggeddata')

In [ ]:
weave.legacy.weave.panels.Board(
    vars={
        # The strange that we need to use two variables for structuring our tag picker and its value.
        'filter_tags': [],
        'tag_picker': lambda filter_tags: weave.legacy.weave.panels.SelectEditor(
            filter_tags,
            choices=weave_internal.const(['a', 'b', 'c'])
        ),
        'data': data,
        'filtered_data': lambda data, filter_tags: data.filter(
            # We don't currently have an intersection op, so we have to write the logic for it here.
            # This will probably not vectorize currently, so will be slower on large amounts of data.
            lambda row: weave.legacy.weave.ops.List.concat(weave.legacy.weave.ops.make_list(a=row['tags'], b=filter_tags)).unique().count()
                < (row['tags'].count() + filter_tags.count())
        )
    },
    panels=[
        weave.legacy.weave.panels.BoardPanel(
            lambda filtered_data: weave.legacy.weave.panels.Table(
                filtered_data,
                columns=[
                    lambda row: row['x'],
                    # Use json_dumps to get a better view for now.
                    lambda row: row['tags'].json_dumps()
                ]
            )
        )
    ]
)